# Dimension-Dependence of MFGS

<span style="font-size:1.2em;">! Run this in **Julia** !</span> 

1. You'll need to install the Julia add on for Jupyter Notebooks (IJulia). Follow guide here: https://datatofish.com/add-julia-to-jupyter/
2. Go to *Kernel* -> *Change Kernel* -> *Julia*

## Packages

In [499]:
include("RCLib.jl") # Include the RC library with all the useful functions
using .RCLib

using LinearAlgebra # For IIL matrix multiplication
using Kronecker
using OrdinaryDiffEq
using DifferentialEquations

## Plotting Packages ##
using CSV # Allows data to be converted to .CSV to use with Python for plotting
using DataFrames # Another useful package for exporting data
using Plots

## Performance Packages ##
using BenchmarkTools

## Temp Lists

In [500]:
T_list = exp10.(range(-3, 4, length=100));

In [501]:
T = [i for i in T_list];
sz3DMFGS_cl = [pred(i) for i in T_list];

## Classical

In [502]:
df_cl = DataFrame(hcat(T, sz3DMFGS_cl), :auto);

In [503]:
# CSV.write("C://Users//crh222//QuantumSpinDynamics//MFGS_data_cl_pred.csv",  df_cl, header = ["T", "sz3D_cl"]);

## Quantum

### Variables

#### Lorentzian Spectral Density

In [504]:
prm1 = [7.0 5.0 10.0];
prm2 = [1.4 0.5 0.16];
prm3 = [1.4 0.001 10.0];
prm4 = [1.4 0 2];

In [505]:
ω0x, Γx, αx = prm4*ωL;
ω0y, Γy, αy = prm4*ωL;
ω0z, Γz, αz = prm4*ωL;

#### RC-Specific

In [506]:
Ωx = ω0x;
λx = ωL*sqrt(αx/Ωx);
δx = Γx/(2π*Ωx);
nx = 3;

In [507]:
Ωy = ω0y;
λy = ωL*sqrt(αy/Ωy);
δy = Γy/(2π*Ωy);
ny = 3;

In [508]:
Ωz = ω0z;
λz = ωL*sqrt(αz/Ωz);
δz = Γz/(2π*Ωz);
nz = 3;

### Gibbs State

In [509]:
ρSpG(T) = gibbs(HSpG, T);

In [510]:
ρSpG(10)

2×2 Matrix{Float64}:
 0.207653  0.0
 0.0       0.792347

In [511]:
tr(ρSpG(10))

1.0

## 1D Mean-Force Gibbs State


In [512]:
Htot1D = HRC1D(nx, λx, Ωx); # System Hamiltonian in original basis

In [513]:
P1D = eigen(Htot1D).vectors; # Find the projectors onto the system eigenbasis

In [514]:
Htot1DT = adjoint(P1D)*Htot1D*P1D; # System Hamiltonian in its eigenbasis

In [515]:
ρtot1D(T) = P1D*gibbs(Htot1DT, T)*adjoint(P1D); # Total Gibbs state in original basis

In [516]:
ρMFGS1D(T) = ptrace(ρtot1D(T), nx); # Trace out the bath

In [517]:
ρMFGS1D(10) # Resulting MFGS

2×2 Matrix{Float64}:
 0.250966     4.33898e-18
 4.33898e-18  0.749034

In [518]:
ρSpG(10) # Compare to the Gibbs state (should be the same when λ = 0)

2×2 Matrix{Float64}:
 0.207653  0.0
 0.0       0.792347

In [519]:
tr(ρMFGS1D(1))

1.0000000000000002

## 2D Mean-Force Gibbs State

In [520]:
Htot2D = HRC2D(nx, nz, λx, λz, Ωx, Ωz); # System Hamiltonian in original basis

In [521]:
P2D = eigen(Htot2D).vectors; # Find the projectors onto the system eigenbasis

In [522]:
Htot2DT = adjoint(P2D)*Htot2D*P2D; # System Hamiltonian in its eigenbasis

In [523]:
ρtot2D(T) = P2D*gibbs(Htot2DT, T)*adjoint(P2D); # Total Gibbs state in original basis

In [524]:
ρMFGS2D(T) = ptrace(ρtot2D(T), nx*nz); # Trace out the bath

In [525]:
ρMFGS2D(10) # Resulting MFGS

2×2 Matrix{Float64}:
 0.243575     2.41953e-16
 2.41953e-16  0.756425

In [526]:
ρSpG(10) # Compare to the Gibbs state (should be the same when λ = 0)

2×2 Matrix{Float64}:
 0.207653  0.0
 0.0       0.792347

In [527]:
tr(ρMFGS2D(1))

1.0

## 3D Mean-Force Gibbs State

In [528]:
Htot3D = HRC3D(nx, ny, nz, λx, λy, λz, Ωx, Ωy, Ωz); # System Hamiltonian in original basis

In [529]:
P3D = eigen(Htot3D).vectors; # Find the projectors onto the system eigenbasis

In [530]:
Htot3DT = adjoint(P3D)*Htot3D*P3D; # System Hamiltonian in its eigenbasis

In [531]:
ρtot3D(T) = P3D*gibbs(Htot3DT, T)*adjoint(P3D); # Total Gibbs state in original basis

In [532]:
ρMFGS3D(T) = ptrace(ρtot3D(T), nx*ny*nz); # Trace out the bath

In [533]:
ρMFGS3D(10) # Resulting MFGS

2×2 Matrix{ComplexF64}:
     0.270319-1.27738e-18im  -3.44166e-17-3.9035e-16im
 -3.44166e-17+3.9035e-16im       0.729681-4.33673e-18im

In [534]:
ρSpG(10) # Compare to the Gibbs state (should be the same when λ = 0)

2×2 Matrix{Float64}:
 0.207653  0.0
 0.0       0.792347

In [539]:
tr(ρMFGS3D(1))

0.9999999999999993 - 1.4221433953446828e-21im

# Plots

In [536]:
expzG(T) = real(tr(ρSpG(T)*sz0)); # Gibbs Sz expectation value
expz1DMFGS(T) = real(tr(ρMFGS1D(T)*sz0)); # 1D MFGS Sz expectation value
expz2DMFGS(T) = real(tr(ρMFGS2D(T)*sz0)); # 2D MFGS Sz expectation value
expz3DMFGS(T) = real(tr(ρMFGS3D(T)*sz0)); # 2D MFGS Sz expectation value

In [537]:
szG_qu = [expzG(i) for i in T_list];
sz1DMFGS_qu = [expz1DMFGS(i) for i in T_list];
sz2DMFGS_qu = [expz2DMFGS(i) for i in T_list];
sz3DMFGS_qu = [expz3DMFGS(i) for i in T_list];
df_qu = DataFrame(hcat(T, szG_qu, sz1DMFGS_qu, sz2DMFGS_qu, sz3DMFGS_qu), :auto);

In [538]:
CSV.write("C://Users//crh222//QuantumSpinDynamics//MFGS_data_qu.csv",  df_qu, header = ["T", "szG", "sz1DMFGS", "sz2DMFGS", "sz3DMFGS"]);